Ideas from Kaggle site:
- What areas of the country are most likely to have UFO sightings?
- Are there any trends in UFO sightings over time? Do they tend to be clustered or seasonal?
- Do clusters of UFO sightings correlate with landmarks, such as airports or government research centers?
- What are the most common UFO descriptions?

New idea:
- Add weather, population... about the sight?
- Military base, airport near the sight?
- 

Some new data links:
- https://www.kaggle.com/sogun3/uspollution

In [1]:
%matplotlib inline

import warnings
import pandas as pd
import numpy as np
import seaborn as sns

### Reading data
- location not found or blank (0.8146%) 
- erroneous or blank time (8.0237%)

In [2]:
# There are some rows with an extra comma that gave reading error
# For now we skip them ~ 300 rows
# TODO: parse warning text and fix them
df = pd.read_csv("../data/complete.csv", error_bad_lines=False, warn_bad_lines=False, low_memory=False)
df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


#### Not useful columns

In [3]:
# date posted seem not useful
df.drop(columns=["date posted"], inplace=True)

# this column is the same as duration (seconds)
df.drop(columns=["duration (hours/min)"], inplace=True)

# Save comment to seperate variable for NLP
comments = df.loc[df.comments.notna(), "comments"]
df.drop(columns=["comments"], inplace=True)

#### Casting column types

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88679 entries, 0 to 88678
Data columns (total 8 columns):
datetime              88679 non-null object
city                  88679 non-null object
state                 81270 non-null object
country               76314 non-null object
shape                 85757 non-null object
duration (seconds)    88677 non-null object
latitude              88679 non-null object
longitude             88679 non-null float64
dtypes: float64(1), object(7)
memory usage: 5.4+ MB


In [5]:
# Column has wrong value in latitude
df = df[df.latitude != '33q.200088']
df["latitude"] = df.latitude.astype(float)

In [6]:
# somehow duration seconds has mixed type float and string 
# -> fixed by using low_memory=False on read_csv
df["duration"] = df["duration (seconds)"].str.replace("`", "")
df["duration"] = df["duration"].astype(np.float32)
df.loc[df["duration"].isna(), "duration"] = df["duration"].mean()
df.drop(columns=["duration (seconds)"], inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88678 entries, 0 to 88678
Data columns (total 8 columns):
datetime     88678 non-null object
city         88678 non-null object
state        81269 non-null object
country      76314 non-null object
shape        85756 non-null object
latitude     88678 non-null float64
longitude    88678 non-null float64
duration     88678 non-null float32
dtypes: float32(1), float64(2), object(5)
memory usage: 5.8+ MB


#### Fill NAs

In [8]:
df[df.country.isna()].shape

(12364, 8)

In [32]:
# TODO:
# Infer state from city, country from state for other countries
df_filled = df.copy()

# Country from state (states as a list)
us_states = np.asarray(['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga',
                        'hi', 'ia', 'id', 'il', 'in', 'ks', 'ky', 'la', 'ma', 'md', 'me',
                        'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj', 'nm',
                        'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx',
                        'ut', 'va', 'vt', 'wa', 'wi', 'wv', 'wy'])

# NaN countries from state
df_filled['country'] = df_filled.apply(lambda row: 'us' if pd.isnull(row['country']) and row['state'] in us_states else row['country'], axis=1)
df_filled[df_filled.country.isna()].shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  del sys.path[0]


(5402, 8)

In [33]:
# Convert datetime column into datetime objects. Time to separate column.
# Some missing or erroneus dates, so using errors='coerce'
df_filled['datetime'] = pd.to_datetime(df_filled['datetime'], errors='coerce')
# TODO: Cast these datetime too
print(df_filled[df_filled.datetime.isna()].shape)
df.loc[df_filled.datetime.isna()].head()

(1220, 8)


,datetime,city,state,country,shape,latitude,longitude,duration
166,10/10/2005 24:00,franklin,in,us,disk,39.480556,-86.055000,0.0
316,10/11/1994 24:00,hot springs and custer,sd,NaN,triangle,43.431646,-103.474362,0.0
417,10/11/2006 24:00,rome,ny,us,oval,43.212778,-75.456111,120.0
487,10/11/2012 24:00,truth or consequences,nm,us,unknown,33.128333,-107.252222,0.0
567,10/1/1972 24:00,sweet home,or,us,unknown,44.397778,-122.735000,0.0


In [35]:
df_filled = df_filled.dropna()
df_filled['hour'] = df_filled['datetime'].dt.hour.astype(int)
df_filled['day'] = df_filled['datetime'].dt.day.astype(int)
df_filled['month'] = df_filled['datetime'].dt.month.astype(int)
df_filled['year'] = df_filled['datetime'].dt.year.astype(int)
df_filled.head()

,datetime,city,state,country,shape,latitude,longitude,duration,hour,day,month,year
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,29.883056,-97.941111,2700.0,20,10,10,1949
1,1949-10-10 21:00:00,lackland afb,tx,us,light,29.384210,-98.581082,7200.0,21,10,10,1949
3,1956-10-10 21:00:00,edna,tx,us,circle,28.978333,-96.645833,20.0,21,10,10,1956
4,1960-10-10 20:00:00,kaneohe,hi,us,light,21.418056,-157.803611,900.0,20,10,10,1960
5,1961-10-10 19:00:00,bristol,tn,us,sphere,36.595000,-82.188889,300.0,19,10,10,1961


In [36]:
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77125 entries, 0 to 88678
Data columns (total 12 columns):
datetime     77125 non-null datetime64[ns]
city         77125 non-null object
state        77125 non-null object
country      77125 non-null object
shape        77125 non-null object
latitude     77125 non-null float64
longitude    77125 non-null float64
duration     77125 non-null float32
hour         77125 non-null int64
day          77125 non-null int64
month        77125 non-null int64
year         77125 non-null int64
dtypes: datetime64[ns](1), float32(1), float64(2), int64(4), object(4)
memory usage: 7.4+ MB


In [37]:
df_filled.to_csv("../data/cleaned.csv", index=False)